# [IAPR][iapr]: Project


**Group ID:** 49

**Author 1 (sciper):** Mariane Brodier (250488)  
**Author 2 (sciper):** Lucas Bost (270139)   
**Author 3 (sciper):** Nicolas Delamaide (271789)   

**Release date:** 07.05.2021  
**Due date:** 03.06.2021 (23h59)


## Important notes

The lab assignments are designed to teach practical implementation of the topics presented during class as well as preparation for the final project, which is a practical project which ties together the topics of the course. 

As such, in the lab assignments/final project, unless otherwise specified, you may, if you choose, use external functions from image processing/ML libraries like opencv and sklearn as long as there is sufficient explanation in the lab report. For example, you do not need to implement your own edge detector, etc.

**! Before handling back the notebook !** rerun the notebook from scratch `Kernel` > `Restart & Run All`


[iapr]: https://github.com/LTS5/iapr

----
## Train our model
#### Import training set

In [1]:
import numpy as np
import os

save_folder = "train_set/"

train_ranks = np.load(save_folder+'train_ranks.npy')
train_suites = np.load(save_folder+'train_suites.npy')

In [2]:
train_im_ranks = train_ranks[:,:-1]
labels_ranks = train_ranks[:,-1].astype(np.int64)

train_im_suites = train_suites[:,:-1]
labels_suites = train_suites[:,-1].astype(np.int64)

#### Train ranks + suites classifiers

In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

def train_MLP(train_images, train_labels):
    
    scaler = StandardScaler() # to scale the values of the pixels
    scaler.fit(train_images)  

    train_images_scaled = scaler.transform(train_images)  # Scale train_images

    hidden_units = 100

    clf = MLPClassifier(hidden_layer_sizes = (hidden_units,), random_state=42, max_iter=150).fit(train_images_scaled, train_labels)
    
    return clf, scaler

In [4]:
ranks_clf, ranks_scaler = train_MLP(train_im_ranks, labels_ranks)
suites_clf, suites_scaler = train_MLP(train_im_suites, labels_suites)

/Users/Nicolas/anaconda3/envs/iapr/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


---
## Import game data


In [7]:
from random import uniform
from skimage import transform as tf

def MNIST_like(im, im_shape, scale_low=1.15, scale_high=1.25, trans_=(-2.2,-3),
                rot=np.deg2rad(2)):
    
    tform = tf.SimilarityTransform(scale=uniform(scale_low, scale_high), translation=trans_,
                                           rotation=uniform(-rot, rot))
    
    return tf.warp(im.reshape(im_shape), tform).flatten()

In [8]:
import re
import pickle
import pandas as pd
from extraction import round_to_elems
from points import assign_cards, assign_points_std, assign_points_adv
import skimage.io

# Change train_games for folder where you put gameX the day of exam
data_base_path = os.path.join(os.pardir, 'project/train_games')

# Predictions for all games and rounds
predictions = []
dealers = []
game_points_std = np.array([0, 0, 0, 0])
game_points_adv = np.array([0, 0, 0, 0])
images = []

# To reorder rounds in predictions
dic_order = {}

for root, dirs, files in os.walk(data_base_path):
    print(root)
    
    loop = 0
    for i in range(len(files)):
        if files[i].endswith(".jpg"):

            # Find the number
            m2 = re.search(r'(\d+)', files[i])
            num_round = None
            if m2:
                num_round = m2.group(0)

            file_im = num_round+".jpg"
            num_round = int(num_round)

            dic_order[num_round-1] = loop
            print(file_im)

            # Current round predictions
            pred_ranks = []
            pred_suites = []

            img = skimage.io.imread(os.path.join(root, file_im))

            ranks, suites, dealer = round_to_elems(img)

            dealers.append(dealer)

            labels_round = []

            # Predict rank + labels
            for idx_player in range(4):

                if (ranks[idx_player].size!=0):

                    # Make rank look like mnist data
                    rank_im = MNIST_like(ranks[idx_player], (28,28))

                    images.append(rank_im)
                    
                    # Scale image
                    rank_scaled = ranks_scaler.transform(rank_im.reshape(1,-1))
                    pred_rank = ranks_clf.predict(rank_scaled)[0]

                else:
                    pred_rank = (-np.float('inf')) # -infinity if no rank found

                pred_ranks.append(pred_rank)

                if (suites[idx_player].size!=0):

                    images.append(suites[idx_player].flatten())
                    
                    # Scale image
                    suite_scaled = suites_scaler.transform(suites[idx_player].flatten().reshape(1,-1))
                    pred_suite = suites_clf.predict(suite_scaled)[0]
                    
                else:
                    pred_suite = None

                pred_suites.append(pred_suite)

            loop += 1

            predictions.append(assign_cards(pred_ranks, pred_suites))
            game_points_std += assign_points_std(pred_ranks)
            game_points_adv += assign_points_adv(pred_ranks, pred_suites, dealer)

predictions = [predictions[dic_order[i]] for i in range(len(predictions))]

../project/train_games
../project/train_games/game1
8.jpg
9.jpg
12.jpg
13.jpg
11.jpg
10.jpg
4.jpg
5.jpg
7.jpg
6.jpg
2.jpg
3.jpg
1.jpg


In [9]:
from utils import print_results

print_results(
    rank_colour=np.asarray(predictions), 
    dealer=dealers, 
    pts_standard=game_points_std.tolist(),
    pts_advanced=game_points_adv.tolist(),
)

The cards played were:
[
['4C', 'JC', 'JD', 'KC'], 
['QD', 'KH', '4C', 'KD'], 
['2S', '7S', '2S', '1D'], 
['JS', '7S', 'KS', 'QS'], 
['0H', '6H', '3H', '2H'], 
['QC', '8C', '2C', '7H'], 
['6H', '2H', '2H', '1H'], 
['Null Null', '8C', '0D', '2C'], 
['4D', '7D', 'QH', '4D'], 
['2C', '4S', '8S', '6S'], 
['2H', 'JH', '6D', '4H'], 
['0C', '7D', '4S', '8S'], 
['2D', '3D', '3D', 'QC'], 
]
Players designated as dealer: [4, 1, 2, 2, 2, 1, 4, 1, 1, 4, 3, 3, 3]
Players points (standard): [2, 5, 3, 4]
Players points (advanced): [2, 7, 4, 1]


The cards played were:
[
['4C', 'JC', 'JD', 'KC'], 
['QD', 'KH', '4C', 'KD'], 
['2S', '7S', '2S', '1D'], 
['JS', '7S', 'KS', 'QS'], 
['0H', '6H', '3H', '2H'], 
['QC', '8C', '2C', '7H'], 
['6H', '2H', '2H', '1H'], 
['7D', '8C', '0D', '2C'], 
['4D', '7D', 'QH', '4D'], 
['2C', '4S', '8S', '6S'], 
['2H', 'JH', '6D', '4H'], 
['0C', '7D', '4S', '8S'], 
['2D', '3D', '3D', 'QC'], 
]
Players designated as dealer: [4, 1, 2, 2, 2, 1, 4, 1, 1, 4, 3, 3, 3]
Players points (standard): [2, 5, 3, 4]
Players points (advanced): [2, 7, 4, 1]